In [19]:
import cv2
import os
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image, ImageDraw
from shapely.geometry import Polygon
from ultralytics import YOLO

# Выбор наилучшей модели

In [25]:
from ensemble_boxes import weighted_boxes_fusion
class IntersectionDetector():
    """Класс для отслеживания пересечений облостей"""

    def __init__(self, model_path) -> None:
        """Метод инициализации"""
        self.model = YOLO(model_path)
        self.image_path = None
        self.zone_path = None

        self.humans_bb = []
        self.zone_bb = []

        self.intersected = []
        self.model()  # сделать первое предсказание

    def find_zone_file(self) -> str:
        """
        Метод получения пути до текстового файла опасной зоны;
        :return путь до текстового файла с опасной зоной;
        """
        arr_dirs = self.image_path.split("/")
        cameras_index = arr_dirs.index("cameras")
        dir_path = self.image_path.split("cameras")[0]

        zone_path = os.path.join(dir_path, "danger_zones", f"danger_{arr_dirs[cameras_index + 1]}.txt")
        return zone_path

    def read_zone_file(self) -> list:
        """
        Метод считывания зоны из текстового файла в список;
        :return список координат из файла зоны;
        """
        zone = []
        with open(self.zone_path) as file:
            for line in file:
                string = line.replace(",", "")
                string = string.replace("[", "")
                string = string.replace("]", "")
                string = string.strip()
                zone.append([int(string.split(" ")[0]), int(string.split(" ")[1])])
        return zone

    @staticmethod
    def get_list_from_xywh(bb: list):
        """
        Форматирование данных из форматы xywh в угловой;
        :param bb - bounding box;
        :return новое представление данных;
        """
        return [[int(bb[0]), int(bb[1])],
                [int(bb[2]), int(bb[1])],
                [int(bb[0]), int(bb[3])],
                [int(bb[2]), int(bb[3])]]

    def format_swap(self, bbx: list) -> list:
        """Метод для перестановки точек в нужной последовательности"""
        # FIXME: Это надо будет обязательно убрать
        bbx_1 = self.get_list_from_xywh(bbx)
        bbx_1_tmp = bbx_1[-1]
        bbx_1[-1] = bbx_1[-2]
        bbx_1[-2] = bbx_1_tmp
        return bbx_1

    def calculate_intersection(self, human: list, draw: bool = False) -> float:
        """
        Метод вычисления площади пересечения;
        :param human - bounding box человека;
        :param draw - режим рисования;
        :return процент тела человека, который пересёкся в опасной зоной;
        """
        square = (human[2] - human[0]) * (human[3] - human[1])
        # Человека переводим в координаты
        human = self.format_swap(human)

        poly_1 = Polygon(human)
        poly_2 = Polygon(self.zone_bb)
        if draw:
            plt.plot(*poly_1.exterior.xy)
            plt.plot(*poly_2.exterior.xy)
            plt.show()

        return (poly_1.intersection(poly_2).area / square) * 100

    def calculate_intersections(self, draw: bool = False) -> list:
        """
        Метод получения тех ситуаций, когда тело входит в зону
        более, чем на 15%;
        :param draw - параметр отрисовки ограничивающих рамок;
        :return список истинных или ложных ситуаций нарушения зоны;
        """
        percentage = 15
        intersections_list = []
        for human in self.humans_bb:
            intersections_list.append(self.calculate_intersection(human, draw))
        # self.show_all()
        self.intersected = [i > percentage for i in intersections_list]
        return intersections_list

    @staticmethod
    def draw_by_points(img: list,
                         zone: list,
                         color: tuple = (255, 0, 0),
                         thick: int = 5) -> list:
        """
        Метод отрисовки точек зоны на изображении;
        :param img - исходное изображение;
        :param zone - координаты зоны;
        :param color - цвет зоны;
        :param thick - толщина линий зоны;
        :return изменённое изображение;
        """
        zone.append(zone[0])
        for p in range(len(zone) - 1):
            cv2.line(img,
                     (zone[p][0], zone[p][1]),
                     (zone[p + 1][0], zone[p + 1][1]),
                     color,
                     thick)

        return img

    def show_final_image(self, image_num) -> list:
        """
        Метод отрисовки всех объектов на изображении;
        :return Отрисованное изображение;
        """
        image = cv2.imread(self.image_path)

        cv2.putText(image, str(image_num), \
                            [100, 200], 
                            cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 255, 255), 10, cv2.LINE_AA)

        i = 0
        for human in self.humans_bb:
            if self.intersected[i]:
                image = cv2.rectangle(image, (int(human[0]), int(human[1])),
                                      (int(human[2]), int(human[3])), (255, 0, 0), 5)
            else:
                image = cv2.rectangle(image, (int(human[0]), int(human[1])),
                                      (int(human[2]), int(human[3])), (0, 255, 0), 5)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.draw_by_points(image, self.zone_bb)

        # plt.imshow(image)
        # plt.show()
        return image

    def compute_metrics(boxes, confidences, weights, iou_thr=0.5, skip_box_thr=0.001):
        """
        Computes WBF metrics for bounding boxes and confidences
        """
        labels = [[0 for _ in conf] for conf in confidences]
        res = weighted_boxes_fusion(boxes, confidences, labels,
                                    weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        return [res[0].tolist(), res[1].tolist()]

    def make_prediction(self) -> None:
        """Метод создания прогноза"""
        pred = self.model(self.image_path)
        if pred[0].boxes.xyxy.tolist():
            self.humans_bb = pred[0].boxes.xyxy.tolist()
        else:
            self.humans_bb = []

    # TODO: А почему не сделать те нижние методы геттером и сеттером? Есть же
    #       @property. А через него можно и остальное сделать
    def set_image_path(self, image_path) -> None:
        """
        Метод установки путей;
        :param image_path - путь до изображений;
        """
        self.image_path = image_path
        self.zone_path = self.find_zone_file()
        self.zone_bb = self.read_zone_file()
        self.make_prediction()

    def get_image_path(self) -> str:
        """
        Метод получения пути до изображений;
        :return - путь до изображения;
        """
        return self.image_path


In [33]:
image_file = "D:/Study/Hack_10.11.2023/cameras/DpR-Csp-uipv-ShV-V1/1a16321d-4371-4721-bbc9-ccfc4a17687c.jpg"
y_true = "D:/Study/Hack_10.11.2023/cameras/DpR-Csp-uipv-ShV-V1/1a16321d-4371-4721-bbc9-ccfc4a17687c.txt"

In [30]:
detector_nano = IntersectionDetector("../models/YOLOv8n.pt")
detector_nano.set_image_path(image_file)

WARNING  'source' is missing. Using 'source=C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 3 persons, 256.6ms
image 2/2 C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 (no detections), 204.4ms
Speed: 6.6ms preprocess, 230.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 D:\Study\Hack_10.11.2023\cameras\DpR-Csp-uipv-ShV-V1\1a16321d-4371-4721-bbc9-ccfc4a17687c.jpg: 384x640 2 persons, 214.4ms
Speed: 4.6ms preprocess, 214.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


In [31]:
detector_medium = IntersectionDetector("../models/YOLOv8m.pt")
detector_medium.set_image_path(image_file)

WARNING  'source' is missing. Using 'source=C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 3 persons, 1124.2ms
image 2/2 C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 (no detections), 846.4ms
Speed: 4.3ms preprocess, 985.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 D:\Study\Hack_10.11.2023\cameras\DpR-Csp-uipv-ShV-V1\1a16321d-4371-4721-bbc9-ccfc4a17687c.jpg: 384x640 2 persons, 855.9ms
Speed: 5.0ms preprocess, 855.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


In [32]:
detector_extra = IntersectionDetector("../models/YOLOv8x.pt")
detector_extra.set_image_path(image_file)

WARNING  'source' is missing. Using 'source=C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets'.

image 1/2 C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 2 persons, 2831.4ms
image 2/2 C:\Users\nikol\AppData\Local\Programs\Python\Python39\Lib\site-packages\ultralytics\assets\zidane.jpg: 384x640 1 person, 2285.7ms
Speed: 6.8ms preprocess, 2558.6ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 D:\Study\Hack_10.11.2023\cameras\DpR-Csp-uipv-ShV-V1\1a16321d-4371-4721-bbc9-ccfc4a17687c.jpg: 384x640 2 persons, 2305.7ms
Speed: 5.0ms preprocess, 2305.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


## Точность

In [48]:
true_val = []
with open(y_true) as file:
    for line in file:
        true_val.append([float(i) for i in line.split()])

In [50]:
detector_extra.humans_bb

[[772.9373168945312, 304.7340087890625, 916.8977661132812, 521.794189453125],
 [1464.29736328125, 300.126953125, 1558.8974609375, 436.6897888183594]]

In [49]:
true_val

[[0.0, 0.788802, 0.342593, 0.047396, 0.137037],
 [0.0, 0.438281, 0.384722, 0.08125, 0.2]]